<a href="https://colab.research.google.com/github/Shivam10816/CS22M082_ASSIGNEMNT_1/blob/q2/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data/255,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data/255,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))


In [43]:
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import numpy as np
import random
from keras.datasets import fashion_mnist

(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data/255,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data/255,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))
class neural_network:

  #it initializes W and b
  def __init__(self,train_data,train_label,hi):
    
    self.train_data=train_data
    self.train_label=train_label
    l= train_data.shape[1]

    self.W =[self.xavier_init(hi[0],l)] 
    self.b =[self.xavier_init(1,hi[0])]
    for i in range(1,len(hi)) :
      self.W.append(self.xavier_init(hi[i],hi[i-1]))
      self.b.append(self.xavier_init(1,hi[i])) 
    self.W.append(self.xavier_init(10,hi[-1]))
    
    self.b.append(self.xavier_init(1,10))
  
  def WX_plus_B(self,W, X, b):
    
    result = np.dot(X, W.transpose())
    row_count = result.shape[0]
    
    row_matrix_repeated = np.tile(b, (row_count, 1))
    return result + row_matrix_repeated

    
  def xavier_init(self,input_size, output_size):
    """
    Initialize weights using Xavier initialization.

    Parameters:
    input_size (int): number of input units.
    output_size (int): number of output units.

    Returns:
    weights (ndarray): array of shape (input_size, output_size) containing the initialized weights.
    """
    # Calculate the variance of the weights
    variance = 2.0 / (input_size + output_size)

    # Calculate the standard deviation of the weights
    standard_deviation = np.sqrt(variance)

    # Generate random weights from a normal distribution with mean 0 and standard deviation standard_deviation
    weights = np.random.normal(loc=0, scale=standard_deviation, size=(input_size, output_size))

    return weights

  #calculates sigmoid for matrix
  def sigmoid(self,x):
  
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
  
  def sum_columns(self,matrix):
    if isinstance(matrix, np.ndarray):
        # if matrix is a numpy array, convert it to a list
        matrix = matrix.tolist()
    
    # sum the elements of each column and store in a list
    column_sums = [sum(col) for col in zip(*matrix)]
    
    # convert the list to a 2D matrix of shape (1 x n)
    row_matrix = np.array([column_sums])
    
    return row_matrix
  #softmax for matrix
  def softmax(self,x):
    # Subtract the maximum value in each row from all the values in that row
    # to prevent numerical instability from very large or very small values
    # in the exponentials of the softmax function.
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / np.sum(e_x, axis=1, keepdims=True)
  
  def subtract_matrices(self,W, W_theta, step_size):
    """
    Subtract the matrices in the second list from the matrices in the first list, after multiplying the matrices in the
    second list by a step size.

    Args:
        first_list (list): A list of numpy arrays representing the first set of matrices.
        second_list (list): A list of numpy arrays representing the second set of matrices.
        step_size (float): The step size to multiply the second set of matrices by.

    Returns:
        list: A list of numpy arrays representing the result of subtracting the second set of matrices from the first set
        of matrices after multiplying the second set of matrices by the step size.
    """
    result_list = []
    for i in range(len(self.W)):
        result = W[i] - step_size * W_theta[i]
        result_list.append(result)
    return result_list

  def sigmoid_derivative(self,matrix):
    """
    Calculate the derivative of the sigmoid function on a 2D matrix.

    Args:
        matrix (numpy.ndarray): A numpy array representing the matrix.

    Returns:
        numpy.ndarray: A numpy array representing the result of calculating the sigmoid derivative on the matrix.
    """
    shift = np.max(matrix, axis=1, keepdims=True)
    exp_matrix = np.exp(matrix - shift)
    sig = 1 / (1 + exp_matrix)
    return sig * (1 - sig)

  def forward_pro(self,X):
    A=[]
    H=[]
    A.append(self.WX_plus_B(self.W[0],X,self.b[0])) # a0 = WoX +bo

    for i in range(1,len(hi)):

      H.append(self.sigmoid(A[i-1])) # hi = g(ai)
      #print(H[i-1])
      A.append( self.WX_plus_B(self.W[i],H[i-1],self.b[i])) # ai = WiX +bi

    H.append(self.sigmoid(A[-2]))
    A.append(self.WX_plus_B(self.W[-1],H[-1],self.b[-1]))
    
    y_hat = self.softmax((A[-1]))
    
    
    
    return A,H,y_hat

  def back_prop(self,X,Y,A,H,y_hat):
    W_theta , b_theta,H_theta,A_theta =self.W,self.b,H,A
    #print(np.argmax(y_hat),lable)
    ey = np.zeros((y_hat.shape[0],y_hat.shape[1]))

    for i in range(0,len(Y)):
      ey[i][Y[i]]=1
  
    L =len(A_theta)
    A_theta[L-1]=(-(ey-y_hat))
    
    #-------------------------
    for k in range(L-1,0,-1):
      
      W_theta[k]=((np.matmul(A_theta[k].transpose(),H[k-1])))/self.batch_size # athetak*h[k-1]
      b_theta[k]=self.sum_columns(A_theta[k])/self.batch_size
      H_theta[k-1]=np.matmul(A_theta[k],self.W[k])
  
      A_theta[k-1]=H_theta[k-1]*self.sigmoid_derivative(A[k-1])

    W_theta[0] = ((np.matmul(A_theta[0].transpose(),X)))
    b_theta[0]=self.sum_columns(A_theta[0])

    # print("-------W-----")
    # for i in W_theta :
    #   print(i)
    # print("-------b-----")
    # for i in b_theta :
    #   print(i)
    # print("-------A-----")
    # for i in A_theta :
    #   print(i)
    # print("-------H-----")
    # for i in H_theta :
    #   print(i)
    
    return W_theta , b_theta

  def accuracy(self, X_test, y_test):
    
    # Feed forward through the network
    A,H,y_hat =self.forward_pro(X_test)
    
    
    y_pred = np.argmax(y_hat, axis=1)
   
    # Calculate accuracy
    acc = np.mean(y_pred == y_test)
    # Calculate accuracy
    

    return acc

  def sgd(self,step_size,batch_size,epoch):
    N = int(len(self.train_data)/batch_size)
    self.batch_size = batch_size
    for e in range(0,epoch):
      start_time = time.time()
      for k in range(0,N):
          minibatch = self.train_data[k*batch_size:min(k*batch_size+batch_size,len(self.train_data))]
          minibatch_lable=self.train_label[k*batch_size:min(k*batch_size+batch_size,len(self.train_data))]
          
          A,H,y_hat=self.forward_pro(minibatch)

          dW,db = self.back_prop(minibatch,minibatch_lable,A,H,y_hat)
          
          self.b =self.subtract_matrices(self.b,db,step_size)
          self.W =self.subtract_matrices(self.W,dW,step_size)

          
          
      print("epoch ",e ,"  %s seconds " % (time.time() - start_time))
     
      
hi=[108,108]   
Net = neural_network(train_data,train_labels,hi)
print("accuracy :-",Net.accuracy(test_data,test_labels)*100,"%")
Net.sgd(0.000001,16,50)
print("accuracy :-",Net.accuracy(test_data,test_labels)*100,"%")

accuracy :- 10.0 %
epoch  0   7.121453046798706 seconds 
epoch  1   5.933155059814453 seconds 
epoch  2   7.5015480518341064 seconds 
epoch  3   6.035588026046753 seconds 
epoch  4   7.403289318084717 seconds 
epoch  5   5.857739210128784 seconds 
epoch  6   7.534205436706543 seconds 
epoch  7   6.048337936401367 seconds 
epoch  8   7.6824116706848145 seconds 
epoch  9   5.81597638130188 seconds 
epoch  10   7.425573825836182 seconds 
epoch  11   5.81052041053772 seconds 
epoch  12   7.5280327796936035 seconds 
epoch  13   5.874327182769775 seconds 
epoch  14   7.220957040786743 seconds 
epoch  15   6.032543182373047 seconds 
epoch  16   7.249237537384033 seconds 
epoch  17   6.6911773681640625 seconds 
epoch  18   6.4222259521484375 seconds 
epoch  19   6.7753074169158936 seconds 
epoch  20   6.438999176025391 seconds 
epoch  21   7.526404857635498 seconds 
epoch  22   5.938875913619995 seconds 
epoch  23   7.358071327209473 seconds 
epoch  24   5.860041856765747 seconds 
epoch  25   

In [33]:
(Net.W[0])

array([[0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.31060806e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:


b =[np.ones(1,50)]

# Net.sgd(0.0001,16,100)
#Net.calculate_accuracy(test_data,test_labels)

In [ ]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = np.reshape(train_data/255,(len(train_data),train_data.shape[1]**2))
test_data = np.reshape(test_data/255,(len(test_data),test_data.shape[1]**2))
test_labels =np.reshape(test_labels,(1,len(test_data)))


In [ ]:
print(test_data.shape)
print(test_labels.shape)